A text similarity classification example for ALBERT.  
The data is from LCQMC.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
%tensorflow_version 2.x

import os
import warnings
import time
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

os.chdir('./drive/My Drive/Python/Research/bert')
warnings.filterwarnings('ignore')

import mymodels as mm

In [2]:
MODEL = 'albert'
MODE = 'cls'
MAXLEN = 128
CATE = 2
DROP = 0.5
DIM = 64
LRATE = 1e-4
BATCH = 64
EPOCH = 5
VOCAB = 'models/albert_tiny_ch/vocab.txt'
CONFIG = 'models/albert_tiny_ch/albert_config_tiny_g.json'
CKPT = 'models/albert_tiny_ch/albert_model.ckpt'

In [3]:
trai_1 = pd.read_csv('tasks/datasets/lcqmc/train.csv')
deve_1 = pd.read_csv('tasks/datasets/lcqmc/dev.csv')
test_1 = pd.read_csv('tasks/datasets/lcqmc/test.csv')
print(trai_1.head())

            sentence1        sentence2  label
0    喜欢打篮球的男生喜欢什么样的女生  爱打篮球的男生喜欢什么样的女生      1
1        我手机丢了，我想换个手机      我想买个新手机，求推荐      1
2            大家觉得她好看吗       大家觉得跑男好看吗？      0
3           求秋色之空漫画全集        求秋色之空全集漫画      1
4  晚上睡觉带着耳机听音乐有什么害处吗？     孕妇可以戴耳机听音乐吗?      0


In [4]:
def data_processing(data, tokenizer, maxlen, batch, training):
  text1, type1, mask1, labe1 = [], [], [], []
  
  for i in range(len(data)):
    if len(data['sentence1'][i])+len(data['sentence2'][i]) > maxlen-2:
      continue

    text2, type2, mask2 = tokenizer.encoding(data['sentence1'][i], data['sentence2'][i], maxlen)
    labe2 = data['label'][i]
    text1.append(text2)
    type1.append(type2)
    mask1.append(mask2)
    labe1.append(labe2)

  text1, type1, mask1, labe1 = np.array(text1), np.array(type1), np.array(mask1), np.array(labe1)
  data1 = tf.data.Dataset.from_tensor_slices((text1, type1, mask1, labe1))
  return data1.shuffle(len(text1)).batch(batch) if training else data1.batch(batch)


toke_1 = mm.Tokenizer()
toke_1.loading(VOCAB)
trai_2 = data_processing(trai_1, toke_1, MAXLEN, BATCH, True)
deve_2 = data_processing(deve_1, toke_1, MAXLEN, BATCH, False)
test_2 = data_processing(test_1, toke_1, MAXLEN, BATCH, False)

In [5]:
class MyModel(keras.Model):
  def __init__(self, model, mode, config, drop, dim, category):
    super(MyModel, self).__init__()
    self.bert = mm.BERT(config, model, mode)
    self.drop = keras.layers.Dropout(drop)
    self.dense1 = keras.layers.Dense(dim, activation='relu')
    self.dense2 = keras.layers.Dense(category, activation='softmax')

  def propagating(self, text, segment, mask, training):
    x1 = self.bert.propagating(text, segment, mask, training)
    x1 = self.drop(x1, training=training)
    return self.dense2(self.dense1(x1))


mode_1 = MyModel(MODEL, MODE, CONFIG, DROP, DIM, CATE)
mode_1.bert.loading(CKPT)
step_1 = EPOCH*(int(len(trai_1)/BATCH)+1)
loss_0 = keras.losses.SparseCategoricalCrossentropy()
opti_1 = mm.AdamWV2(step_1, LRATE)

In [6]:
loss_1 = tf.keras.metrics.Mean(name='training_loss')
accu_1 = tf.keras.metrics.SparseCategoricalAccuracy(name='training_accuracy')
loss_2 = tf.keras.metrics.Mean(name='dev_loss')
accu_2 = tf.keras.metrics.SparseCategoricalAccuracy(name='dev_accuracy')


@tf.function
def step_training(text, segment, mask, y):
  with tf.GradientTape() as tape_1:
    pred_1 = mode_1.propagating(text, segment, mask, True)
    valu_1 = loss_0(y, pred_1)

  grad_1 = tape_1.gradient(valu_1, mode_1.trainable_variables)
  grad_1, _ = tf.clip_by_global_norm(grad_1, 1.0)
  opti_1.apply_gradients(zip(grad_1, mode_1.trainable_variables))
  loss_1(valu_1)
  accu_1(y, pred_1)


@tf.function
def step_evaluating(text, segment, mask, y):
  pred_1 = mode_1.propagating(text, segment, mask, False)
  valu_1 = loss_0(y, pred_1)
  loss_2(valu_1)
  accu_2(y, pred_1)

In [7]:
temp_1 = 'Training loss is {}, accuracy is {}, and step cost is {}.'
temp_2 = 'Dev accuracy is {}, and epoch cost is {}.'
coun_1 = 0

for e_1 in range(EPOCH):
  print('Epoch {} running.'.format(e_1+1))
  time_0 = time.time()

  for x_1, x_2, x_3, y_1 in trai_2:
    time_1, coun_1 = time.time(), coun_1+1
    step_training(x_1, x_2, x_3, y_1)

    if coun_1 % 1000 == 0:
      o_1, o_2 = round(float(loss_1.result()), 4), round(float(accu_1.result()), 4)
      print(temp_1.format(o_1, o_2, round(time.time()-time_1, 4)))

  for x_1, x_2, x_3, y_1 in deve_2:
    step_evaluating(x_1, x_2, x_3, y_1)

  print(temp_2.format(round(float(accu_2.result()), 4), round(time.time()-time_0, 4)))
  print('**********')
  accu_1.reset_states()
  accu_2.reset_states()

Epoch 1 running.
Training loss is 0.3516, accuracy is 0.8455, and step cost is 0.1059.
Training loss is 0.3072, accuracy is 0.8674, and step cost is 0.111.
Training loss is 0.2875, accuracy is 0.8768, and step cost is 0.1131.
Dev accuracy is 0.7988, and epoch cost is 413.6534.
**********
Epoch 2 running.
Training loss is 0.2732, accuracy is 0.9094, and step cost is 0.1099.
Training loss is 0.2612, accuracy is 0.9118, and step cost is 0.1134.
Training loss is 0.2523, accuracy is 0.9133, and step cost is 0.1098.
Training loss is 0.2458, accuracy is 0.9137, and step cost is 0.1171.
Dev accuracy is 0.8409, and epoch cost is 413.5434.
**********
Epoch 3 running.
Training loss is 0.2385, accuracy is 0.9311, and step cost is 0.1075.
Training loss is 0.2314, accuracy is 0.9312, and step cost is 0.1091.
Training loss is 0.2256, accuracy is 0.9313, and step cost is 0.1128.
Training loss is 0.2205, accuracy is 0.9313, and step cost is 0.1073.
Dev accuracy is 0.8434, and epoch cost is 408.0028.
**

In [8]:
c_1, c_2 = 0, 0

for x_1, x_2, x_3, y_1 in test_2:
  pred_1 = mode_1.propagating(x_1, x_2, x_3, False)
  comp_1 = sum(np.array(y_1)==np.argmax(pred_1, 1))
  c_1, c_2 = c_1+len(pred_1), c_2+comp_1

print('Test accuracy is '+str(round(float(c_2/c_1), 4))+'.')

Test accuracy is 0.852.
